In [5]:
# Add project root to Python path
import os
import sys
from pathlib import Path

# Get the project root directory (parent of notebooks directory)
project_root = Path(os.path.abspath('')).parent

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"Added {project_root} to Python path")

Added /Users/anassyed/Documents/lynqo to Python path


In [6]:
# Standard imports
import cv2
import os
import numpy as np
from roboflow import Roboflow
import random
import shutil
import torch
from ultralytics import YOLO
from pathlib import Path

# Import utility functions
from src.utils import predict_image
from src.utils import display_surgical_detections
from src.utils import visualize_detections

In [7]:
PATH_TO_DADASET = Path("../datasets/Surgical-Tools-Detection-1")
if not PATH_TO_DADASET.exists():
    rf = Roboflow(api_key="P9o0JjFVWdK2Z3nzefUI")
    project = rf.workspace("northeastern-university-yu4fz").project("surgical-tools-detection-c8b3w")
    version = project.version(1)
    dataset = version.download("yolov11")
else:
    print("Dataset folder already exists, skipping download.")
                

Dataset folder already exists, skipping download.


In [8]:
os.getcwd()

'/Users/anassyed/Documents/lynqo/notebooks'

In [9]:
# getting the instrument names from config.json 
config_path = '../config.json'
with open(config_path, 'r') as f:
    config = f.read()
    config = eval(config)  # Convert string to dictionary
    instrument_names = config['SURGICAL_INSTRUMENTS']
    print("Instrument names:", instrument_names)


Instrument names: {'0': 'Mayo-Hegar needle holder', '1': 'Adson tissue forceps (plain thumb forceps)', '2': 'Sponge-holding (Foerster) forceps', '3': 'Kelly hemostat (straight)', '4': 'DeBakey tissue forceps', '5': 'Mosquito hemostat (straight)', '6': 'Weitlaner self-retaining retractor', '7': 'Surgical scissors (e.g. Metzenbaum/Mayo scissors)', '8': 'Kelly hemostat (curved)', '9': 'Mixter (right-angle) hemostat', '10': 'Bone hook', '11': 'Lister (bandage) scissors', '12': 'Tissue forceps (dressing forceps)', '13': 'Kocher (Ochsner) forceps', '14': 'Frazier suction tip', '15': 'Electrocautery ("Bovie") pencil', '16': 'Scalpel (No. 3 handle with blade)', '17': 'Kelly hemostat (long)'}


After augmentation

In [14]:
# Setup paths for original and augmented data
train_img_dir = PATH_TO_DADASET / 'train' / 'images'
train_aug_dir = PATH_TO_DADASET / 'train' / 'images_aug'
train_lbl_dir = PATH_TO_DADASET / 'train' / 'labels'
train_lbl_aug_dir = PATH_TO_DADASET / 'train' / 'labels_aug'

# Get all images (both original and augmented)
all_images = list(train_img_dir.glob('*.jpg'))
all_aug_images = list(train_aug_dir.glob('*.jpg'))
total_images = len(all_images) + len(all_aug_images)
subset_size = int(total_images * 0.25)  # 25% of total

print(f"Original images: {len(all_images)}")
print(f"Augmented images: {len(all_aug_images)}")
print(f"Total images: {total_images}")


Original images: 17789
Augmented images: 35417
Total images: 53206


In [17]:
# Create subset directories
subset_size = int(total_images * 0.10) # 10% of total
subset_dir = PATH_TO_DADASET / 'train_subset'
subset_img_dir = subset_dir / 'images'
subset_lbl_dir = subset_dir / 'labels'

subset_img_dir.mkdir(parents=True, exist_ok=True)
subset_lbl_dir.mkdir(parents=True, exist_ok=True)

# Randomly select from both original and augmented
random.seed(42)  # for reproducibility
combined_images = all_images + all_aug_images
selected_images = random.sample(combined_images, subset_size)

In [18]:
# Copy selected files
for img_path in selected_images:
    # Copy image
    shutil.copy2(img_path, subset_img_dir)
    
    # Determine correct label directory based on image source
    if 'images_aug' in str(img_path):
        lbl_dir = train_lbl_aug_dir
    else:
        lbl_dir = train_lbl_dir
        
    # Copy corresponding label
    lbl_path = lbl_dir / f"{img_path.stem}.txt"
    if lbl_path.exists():
        shutil.copy2(lbl_path, subset_lbl_dir)

print(f"\nCreated subset dataset at: {subset_dir}")


Created subset dataset at: ../datasets/Surgical-Tools-Detection-1/train_subset


In [19]:
# Create modified yaml for subset training
import yaml

yaml_path = PATH_TO_DADASET / 'data.yaml'
subset_yaml_path = PATH_TO_DADASET / 'data_subset.yaml'

with open(yaml_path) as f:
    yaml_data = yaml.safe_load(f)

# Update train path to use subset
yaml_data['train'] = str(subset_img_dir.relative_to(PATH_TO_DADASET))

with open(subset_yaml_path, 'w') as f:
    yaml.dump(yaml_data, f)

In [20]:
# Initialize YOLO model for training
model = YOLO("yolo11n.pt")


In [24]:
use_subset = True  # Set to True to use the subset dataset
if use_subset:
    data_path = subset_yaml_path
else:
    data_path = yaml_path

In [25]:

# Train the model using the dataset
model.train(
    data=str(data_path),  # path to data yaml file using PATH_TO_DADASET
    epochs=5,                # number of epochs
    imgsz=416,                # image size
    batch=16,                 # batch size
    patience=50,              # early stopping patience
    verbose=True             # print training progress
)

Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.7.0 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/Surgical-Tools-Detection-1/data_subset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.0, pret

train: Scanning /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/labels... 5320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5320/5320 [00:00<00:00, 6854.49it/s]

train: /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/images/Add_labelled00295_jpg.rf.a6fad8291ea70bd9ffd28b1e51de1838_aug0.jpg: 1 duplicate labels removed
train: /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/images/Add_labelled00519_jpg.rf.7d55bd7f4c6161616bb9ff9c81173677_aug0.jpg: 1 duplicate labels removed
train: /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/images/Add_labelled00695_jpg.rf.4e8710dc23e98c473cc20402c9eccd1c_aug1.jpg: 1 duplicate labels removed


train: New cache created: /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 55.9±50.7 MB/s, size: 10.1 KB)


val: Scanning /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/valid/labels.cache... 1980 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1980/1980 [00:00<?, ?it/s]


...

                   all       1980       2551      0.242      0.526      0.285      0.235



5 epochs completed in 1.849 hours.
Optimizer stripped from /Users/anassyed/Documents/lynqo/runs/detect/train9/weights/last.pt, 5.4MB
Optimizer stripped from /Users/anassyed/Documents/lynqo/runs/detect/train9/weights/best.pt, 5.4MB

Validating /Users/anassyed/Documents/lynqo/runs/detect/train9/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.7.0 CPU (Apple M2)
YOLO11n summary (fused): 100 layers, 2,585,662 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [02:08<00:00,  2.07s/it]


                   all       1980       2551      0.241      0.528      0.284      0.235
                     0        176        195      0.206      0.769      0.287      0.261
                     1         74        102      0.231      0.549      0.211      0.172
                    10         92        105      0.446      0.486       0.48       0.33
                    11        172        197      0.312      0.772      0.695      0.575
                    12        100        127      0.166     0.0866      0.187      0.157
                    13        109        117      0.223      0.735      0.337      0.298
                    14         96        113      0.188      0.469      0.144      0.129
                    15        107        107      0.402      0.991      0.511      0.347
                    16         80         80      0.266          1      0.339      0.248
                    17         94         94      0.308      0.362      0.266      0.256
                     

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x31ca9e990>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.0

In [11]:
# Load YOLO model
MODEL_PATH = Path("../runs/detect/train9/weights/best.pt")
# loaded_model = YOLO(MODEL_PATH)
loaded_model = YOLO(MODEL_PATH)

In [12]:
result = predict_image(loaded_model, 
              '../datasets/stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg')

2025-06-01 12:12:00.697 python[10528:1586784] +[IMKClient subclass]: chose IMKClient_Modern
2025-06-01 12:12:00.697 python[10528:1586784] +[IMKInputSession subclass]: chose IMKInputSession_Modern


image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg: 320x416 1 11, 1 2, 75.8ms
Speed: 6.1ms preprocess, 75.8ms inference, 6.3ms postprocess per image at shape (1, 3, 320, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict2
1 label saved to /Users/anassyed/Documents/lynqo/runs/detect/predict2/labels


In [14]:
display_surgical_detections(result, instrument_names)


Detected Surgical Instruments in stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg:
--------------------------------------------------
Summary of Detected Instruments:
Bone hook: 1 instance
Kelly hemostat (straight): 1 instance
--------------------------------------------------

Detailed Detection Information:
Instrument: Bone hook
Confidence: 13.54%
Location (x1,y1,x2,y2): [370.6, 485.2, 986.2, 789.5]
--------------------------------------------------
Instrument: Kelly hemostat (straight)
Confidence: 13.00%
Location (x1,y1,x2,y2): [411.2, 493.0, 938.7, 888.9]
--------------------------------------------------


In [15]:
result = predict_image(loaded_model,
              '../datasets/General-Surgical-Instruments-What-Are-They-And-What-Are-They-Used-For-Medilogic-21564597.jpg',
              conf_threshold=0.1, save=True, show=True)


image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/General-Surgical-Instruments-What-Are-They-And-What-Are-They-Used-For-Medilogic-21564597.jpg: 320x416 1 5, 54.0ms
Speed: 2.7ms preprocess, 54.0ms inference, 5.9ms postprocess per image at shape (1, 3, 320, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict2
2 labels saved to /Users/anassyed/Documents/lynqo/runs/detect/predict2/labels


In [17]:
# Test the new function with surgical instrument names
print("Displaying detections with proper instrument names:")
display_surgical_detections(result, instrument_names)

Displaying detections with proper instrument names:

Detected Surgical Instruments in General-Surgical-Instruments-What-Are-They-And-What-Are-They-Used-For-Medilogic-21564597.jpg:
--------------------------------------------------
Summary of Detected Instruments:
Kocher (Ochsner) forceps: 1 instance
--------------------------------------------------

Detailed Detection Information:
Instrument: Kocher (Ochsner) forceps
Confidence: 10.22%
Location (x1,y1,x2,y2): [1127.7, 270.7, 1597.7, 872.9]
--------------------------------------------------


In [87]:
# Test the updated function with proper instrument counting
print("Displaying detections with instrument counts:")
display_surgical_detections(result)

Displaying detections with instrument counts:

Detected Surgical Instruments in General-Surgical-Instruments-What-Are-They-And-What-Are-They-Used-For-Medilogic-21564597.jpg:
--------------------------------------------------
Summary of Detected Instruments:
Kocher (Ochsner) forceps: 1 instance
--------------------------------------------------

Detailed Detection Information:
Instrument: Kocher (Ochsner) forceps
Confidence: 10.22%
Location (x1,y1,x2,y2): [1127.7, 270.7, 1597.7, 872.9]
--------------------------------------------------


In [19]:
# Test with another image
result = predict_image(loaded_model,
              '../datasets/stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg',
              conf_threshold=0.1, save=True, show=True)

# Display the results
print("\nDisplaying detections for new image:")
display_surgical_detections(result, instrument_names)


image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg: 320x416 1 11, 1 2, 49.0ms
Speed: 2.0ms preprocess, 49.0ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict2
2 labels saved to /Users/anassyed/Documents/lynqo/runs/detect/predict2/labels

Displaying detections for new image:

Detected Surgical Instruments in stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg:
--------------------------------------------------
Summary of Detected Instruments:
Bone hook: 1 instance
Kelly hemostat (straight): 1 instance
--------------------------------------------------

Detailed Detection Information:
Instrument: Bone hook
Confidence: 13.54%
Location (x1,y1,x2,y2): [370.6, 485.2, 986.2, 789.5]
--------------------------------------------------
Instrument: Kelly hemostat (straight)
Confidence: 13.00%
Location (x1,y1,x2,y2): [411

In [12]:
result = predict_image(loaded_model,
              '../datasets/sets.jpg',
              conf_threshold=0.1, save=True, show=True)

2025-06-01 12:46:31.691 python[19756:1627198] +[IMKClient subclass]: chose IMKClient_Modern
2025-06-01 12:46:31.691 python[19756:1627198] +[IMKInputSession subclass]: chose IMKInputSession_Modern


image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/sets.jpg: 384x416 9 11s, 4 5s, 1 6, 8 7s, 1 9, 57.0ms
Speed: 3.5ms preprocess, 57.0ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict3
1 label saved to /Users/anassyed/Documents/lynqo/runs/detect/predict3/labels
Speed: 3.5ms preprocess, 57.0ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict3
1 label saved to /Users/anassyed/Documents/lynqo/runs/detect/predict3/labels


In [13]:
display_surgical_detections(result, instrument_names)


Detected Surgical Instruments in sets.jpg:
--------------------------------------------------
Summary of Detected Instruments:
Electrocautery ("Bovie") pencil: 8 instances
Kelly hemostat (straight): 9 instances
Frazier suction tip: 1 instance
Kocher (Ochsner) forceps: 4 instances
Kelly hemostat (long): 1 instance
--------------------------------------------------

Detailed Detection Information:
Instrument: Electrocautery ("Bovie") pencil
Confidence: 53.96%
Location (x1,y1,x2,y2): [263.3, 230.5, 362.2, 384.1]
--------------------------------------------------
Instrument: Kelly hemostat (straight)
Confidence: 40.76%
Location (x1,y1,x2,y2): [152.9, 9.1, 241.7, 208.6]
--------------------------------------------------
Instrument: Electrocautery ("Bovie") pencil
Confidence: 32.87%
Location (x1,y1,x2,y2): [281.6, 22.7, 368.2, 187.5]
--------------------------------------------------
Instrument: Kelly hemostat (straight)
Confidence: 32.16%
Location (x1,y1,x2,y2): [222.6, 53.5, 250.2, 228.9]

In [14]:
result = predict_image(loaded_model,
              '../datasets/18-14-pcs-minor-surgery-set-surgical-instruments-set-arineo-original-imagbgakxvkwvudg.jpg',
              conf_threshold=0.1, save=True, show=True)


image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/18-14-pcs-minor-surgery-set-surgical-instruments-set-arineo-original-imagbgakxvkwvudg.jpg: 352x416 4 11s, 1 12, 2 5s, 4 7s, 101.4ms
Speed: 18.4ms preprocess, 101.4ms inference, 10.5ms postprocess per image at shape (1, 3, 352, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict3
2 labels saved to /Users/anassyed/Documents/lynqo/runs/detect/predict3/labels
image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/18-14-pcs-minor-surgery-set-surgical-instruments-set-arineo-original-imagbgakxvkwvudg.jpg: 352x416 4 11s, 1 12, 2 5s, 4 7s, 101.4ms
Speed: 18.4ms preprocess, 101.4ms inference, 10.5ms postprocess per image at shape (1, 3, 352, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict3
2 labels saved to /Users/anassyed/Documents/lynqo/runs/detect/predict3/labels


In [ ]:
display_surgical_detections(result, instrument_names)

{'detected_instruments': [{'type': 'Kelly hemostat (straight)', 'count': 4},
  {'type': 'Electrocautery ("Bovie") pencil', 'count': 4},
  {'type': 'Kocher (Ochsner) forceps', 'count': 2},
  {'type': 'DeBakey tissue forceps', 'count': 1}]}